<a href="https://colab.research.google.com/github/beatlesatani/coding_september/blob/main/web_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install flask-ngrok

In [2]:
!pip install tensorflow_text
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.2 MB/s eta 0:00:00


In [3]:
from transformers import BertTokenizer,TFBertModel,BertConfig,TFBertForSequenceClassification
import tensorflow as tf
import torch
import numpy as np
from tensorflow import keras
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template
import pickle
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [5]:
model_path = '/drive/MyDrive/sentiment_analysis/bert_model.tf'
# load the model
loaded_model = keras.models.load_model(model_path)

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case = True)

In [7]:
test_sentence = ['i am so happy']

In [8]:
tf_batch = tokenizer(test_sentence, max_length = 75, padding=True, truncation=True, return_tensors='tf')

In [9]:
tf_outputs = loaded_model(tf_batch)

In [10]:
tf_outputs

{'logits': <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-2.0649917, -0.8902506,  3.5346541]], dtype=float32)>}

In [11]:
logits = tf_outputs['logits']

# Apply softmax to the logits to get probabilities
probabilities = tf.nn.softmax(logits, axis=-1)
print(probabilities)

# Get the predicted class with the highest probability
predicted_class = np.argmax(probabilities, axis=-1)

# The predicted_class variable now contains the predicted sentiment label (0, 1, or 2)
print("Predicted Sentiment:", predicted_class[0])

tf.Tensor([[0.00364209 0.01179054 0.9845674 ]], shape=(1, 3), dtype=float32)
Predicted Sentiment: 2


In [12]:
# i tried to use my original tokenizer from final.ipnyb.
# but by using this I got lots of error, like score is beyond the range of [0,1,2] like 402
""""def mask_inputs_for_bert(tweets, max_len):
  # Encode the tweet using the tokenizer.
  encoded_dict = tokenizer.encode_plus(
            tweets,
            add_special_tokens=True, #add cls and sep
            max_length=max_len,   #pad & truncate all sentences
            pad_to_max_length=True,
            return_attention_mask=True  #construct attention_masks
        )
  input_id = encoded_dict['input_ids']
  attention_mask =encoded_dict['attention_mask']

  # Convert the lists to tensors (vectors) and return
  input_id = tf.convert_to_tensor(input_id)
  attention_mask = tf.convert_to_tensor(attention_mask)

  return input_id, attention_mask"""

'"def mask_inputs_for_bert(tweets, max_len):\n  # Encode the tweet using the tokenizer.\n  encoded_dict = tokenizer.encode_plus(\n            tweets,\n            add_special_tokens=True, #add cls and sep\n            max_length=max_len,   #pad & truncate all sentences\n            pad_to_max_length=True,\n            return_attention_mask=True  #construct attention_masks\n        )\n  input_id = encoded_dict[\'input_ids\']\n  attention_mask =encoded_dict[\'attention_mask\']\n\n  # Convert the lists to tensors (vectors) and return\n  input_id = tf.convert_to_tensor(input_id)\n  attention_mask = tf.convert_to_tensor(attention_mask)\n\n  return input_id, attention_mask'

In [13]:
# put them into one function
def sentiment_analysis (text):
  tf_batch = tokenizer(text, max_length = 75, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = loaded_model(tf_batch)
  logits = tf_outputs['logits']
  # Apply softmax to the logits to get probabilities
  probabilities = tf.nn.softmax(logits, axis=-1)
  # Get the predicted class with the highest probability
  predicted_class = np.argmax(probabilities, axis=-1)
  if (predicted_class == 0):
    return 'Negative'
  elif (predicted_class == 1):
    return "Neutral"
  else:
    return "Positive"

In [24]:
text = "i'm eating apples"
print(sentiment_analysis(text))
#the funciton works

Neutral


In [30]:
import getpass

from pyngrok import ngrok
from pyngrok import conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")


ModuleNotFoundError: ignored

In [26]:
from flask import Flask, request, render_template

app = Flask(__name__)
run_with_ngrok(app)

# ホームページ
@app.route('/')
def home():
    return render_template('index.html')

# 感情分析
@app.route('/analyze', methods=['POST'])
def analyze():
    text = request.form['text']
    sentiment = sentiment_analysis(text)
    return render_template('index.html', text=text, sentiment=sentiment)


app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c142-35-229-76-43.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-26-f74b8b19e8b6>", line 9, in home
    return render_template('index.html')
  File "/usr/local/lib/python3.10/dist-packages/flask/templating.py", line 146, in render_template
    template = app.jinja_env.get_or_select_template(template_name_or_list)
  File "/usr/local/lib/python3.10/dist-packages/jinja2/environment.py", line 1081, in

In [ ]:
x